In [26]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import joblib
from datetime import datetime

In [4]:
data = pd.read_csv('../data/stats.csv')
cases = pd.read_csv('../data/cases_labeled.csv', sep=';')
data

,Герой,Тип поручения,Роль,Оценка за качество,Оценка по срокам,Оценка за вежливость,Затрачено часов,Затрачено дней,Сумма вознаграждения,Количество поручений
0,Агата,0,лучник,4.125000,4.375000,4.125000,0.666667,4.250000,16000.000000,8
1,Агата,1,следопыт,4.000000,3.500000,3.500000,4.000000,6.250000,15500.000000,4
2,Агата,2,следопыт,3.777778,3.777778,4.000000,2.500000,5.555556,16944.444444,18
3,Агата,3,лучник,3.888889,3.111111,4.222222,0.666667,5.222222,8888.888889,9
4,Агата,3,рейнджер,3.857143,3.000000,4.000000,2.000000,4.714286,8642.857143,7
...,...,...,...,...,...,...,...,...,...,...
81,Юлия,2,следопыт,3.923077,3.615385,3.846154,3.750000,6.230769,15653.846154,26
82,Юлия,3,лекарь,3.833333,3.750000,4.083333,18.000000,6.666667,14041.666667,12
83,Юлия,3,мечник,3.818182,3.772727,4.045455,1.000000,6.318182,14863.636364,22
84,Юлия,3,рейнджер,3.833333,3.888889,4.055556,0.666667,6.944444,15861.111111,18


In [5]:
roles_for_tasks = {
    0: ['лекарь', 'лучник', 'мечник', 'боевой маг'],
    1: ['следопыт'],
    2: ['следопыт'],
    3: ['рейнджер', 'следопыт', 'лекарь', 'лучник', 'мечник', 'боевой маг']
}

In [6]:
def compute_score(hero, task_type, salary=0, weight_quality=1, weight_time=1, weight_politeness=1, salary_weight = 500):
    """
    Функция для вычисления "стоимости" героя для выполнения конкретного поручения с учетом
    типа поручения и роли героя.
    
    Параметры:
    - hero: строка с данными героя (Series).
    - task_type: тип поручения.
    - roles_for_tasks: словарь с ролями для каждого типа поручения.
    - weight_quality, weight_time, weight_politeness: веса для оценки параметров.
    
    Возвращает:
    - Оценку пригодности героя для выполнения поручения.
    """
    
    # Проверим, соответствует ли роль героя типу поручения
    if len(set(hero['Роль'].unique()) & set(roles_for_tasks[task_type])) == 0:
        return 0  # Если роль героя не подходит для этого поручения, возвращаем 0 (герой не подходит)

    # Оценки для конкретного героя и типа поручения
    quality = hero['Оценка за качество'].sum()
    
    time1 = hero['Затрачено часов'].sum()
    time2 = hero['Затрачено дней'].sum()
    speed = hero['Оценка по срокам'].sum()
    politeness = hero['Оценка за вежливость'].sum()
    
    # Вычисляем стоимость героя с учетом всех факторов
    score = (weight_quality * quality) + (speed * weight_time / time1 / time2) + (weight_politeness * politeness)
    return score


# Пример: вычисление для героя
task_type = 0  # Тип поручения
hero = data.loc[(data['Герой'] == 'Агата') & (data['Тип поручения'] == task_type)]


print(compute_score(hero, task_type))


9.794117646286765


In [7]:
task_type = 0  # Например, тип поручения 0
heroes = data['Герой'].unique()
# Пример: вычисление для героя
for name in heroes:
    hero = data.loc[(data['Герой'] == name) & (data['Тип поручения'] == task_type)]
    for role in hero['Роль'].to_list():
        score = compute_score(hero.loc[hero['Роль'] == role], task_type)
        print(f"Герой: {name}, Роль: {role}, Оценка: {score}")


Герой: Агата, Роль: лучник, Оценка: 9.794117646286765
Герой: Альфред, Роль: лекарь, Оценка: 7.134979423868312
Герой: Альфред, Роль: мечник, Оценка: 7.4148148148148145
Герой: Бендер, Роль: мечник, Оценка: 9.239071037673497
Герой: Бенедикт, Роль: лекарь, Оценка: 8.145390070921986
Герой: Бенедикт, Роль: мечник, Оценка: 8.436170212765958
Герой: Глюкоза, Роль: боевой маг, Оценка: 8.798295453873578
Герой: Леопольд, Роль: мечник, Оценка: 8.54945054945055
Герой: Мартин, Роль: лекарь, Оценка: 8.63968253968254
Герой: Мартин, Роль: лучник, Оценка: 9.314285714285715
Герой: Пастушок, Роль: мечник, Оценка: 8.983739836739836
Герой: Синеглазый, Роль: лекарь, Оценка: 7.493015873015873
Герой: Синеглазый, Роль: мечник, Оценка: 8.588571428571429
Герой: Соня, Роль: боевой маг, Оценка: 8.511864406779662
Герой: Соня, Роль: лекарь, Оценка: 7.879096045197739
Герой: Фредерик, Роль: лекарь, Оценка: 7.83939393939394
Герой: Фредерик, Роль: лучник, Оценка: 8.50909090909091
Герой: Юлия, Роль: лекарь, Оценка: 7.67891

In [8]:
cases['Нормированная сумма'] = (cases['Сумма вознаграждения'] - cases['Сумма вознаграждения'].min()) / (cases['Сумма вознаграждения'].max() - cases['Сумма вознаграждения'].min())

In [31]:
# Для каждого типа поручения решим задачу на подбор героев
def optimize_team_for_task(task, heroes):
    # Формируем список оценок для каждого героя по данному поручению
    salary = task['Нормированная сумма']
    task_type = task['Тип поручения']
    
    scores = []
    for name in heroes:
        hero = data.loc[(data['Герой'] == name) & (data['Тип поручения'] == task_type)]
        score = []
        for role in hero['Роль'].to_list():
            score.append(compute_score(hero.loc[hero['Роль'] == role], task_type))
        scores.append(max(score))

    # задача сводится к нахождению героев с наибольшими баллами
    # Используем минимизацию, поэтому берём отрицательный score
    c = [-score for score in scores]
    
    # Ограничения на количество героев: 1-4 героя
    A_eq = np.ones((1, len(heroes)))

    if salary < 0.25:
        team_len = 1
    elif salary < 0.5:
        team_len = min(2, len(heroes))
    elif salary < 0.75:
        team_len = min(3, len(heroes))
    else:
        team_len = min(4, len(heroes))

    b_eq = [team_len]  # Не более 4 героев
    x_bounds = [(0, 1) for _ in range(len(heroes))]  # Каждому герою либо 0, либо 1 (выбираем героя или нет)

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

    # Получаем выбранных героев
    selected_heroes = [heroes[i] for i in range(len(res.x)) if res.x[i] > 0.5]
    #return selected_heroes[max(selected_heroes.keys())]
    return selected_heroes

# Пример
free_heroes = heroes.copy()
team_for_task_0 = optimize_team_for_task(cases.iloc[0], free_heroes)
print("Оптимальная команда для поручения типа:", team_for_task_0)


Оптимальная команда для поручения типа: ['Бендер']


### **Демонстрация работы на не выполненных заказах**

#### Загрузка данных и определение классов поручений

- Количество членов команды решено формировать в связи с величиной вознаграждения (т.к. принято допущение, что наличие в тексте упоминания "опасности" не отражается на оплате заказа, а также на времени его выполнения) 
- Команды формируются по взвешенной оценке, учитывающей среднее время выполнения заказов, средние оценки
- Герои возвращаются в список доступных в связи по истечении количества дней = max(количество дней на заказ)

In [35]:
cur_cases = pd.read_csv('../data/cases.csv', sep=';')
cur_cases.rename(columns={'\ufeffНомер поручения':'Номер поручения'}, inplace=True)
cur_cases = cur_cases.loc[cur_cases['Выполнено'] == 'нет'].sort_values(by='Дата поручения')

In [11]:
model = joblib.load('../models/classification_model.pkl')

In [36]:
cur_cases['Нормированная сумма'] = (cur_cases['Сумма вознаграждения'] - 5000) / (25000)
cur_cases['Тип поручения'] = model.predict(cur_cases['Описание'])
cur_cases.head()

,Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание,Нормированная сумма,Тип поручения
134,11134,Иван,1053-09-04,нет,NaN,NaN,27500,В пещере появвилось огромное каменное чудовище...,0.90,0
56,11056,Мария,1053-09-06,нет,NaN,NaN,23500,По дороге из деревни у меня пропала драгоценно...,0.74,2
381,11381,Егор,1053-09-20,нет,NaN,NaN,7000,В пещере появвилось огромное каменное чудовище...,0.08,0
311,11311,Бабушка Синь,1053-09-20,нет,NaN,NaN,19000,По дороге из деревни у меня была украдена драг...,0.56,2
417,11417,Леонтия,1053-09-22,нет,NaN,NaN,20500,Недалеко от города у меня была украдена драгоц...,0.62,2


In [44]:
max_time_interval = cases['Затрачено дней'].max()
teams = []
free_heroes = set(heroes)
query = []
for idx, row in cur_cases.iterrows():
    for date, team in query:
        if (datetime.strptime(row['Дата поручения'], '%Y-%m-%d') - date).days >= max_time_interval:
            free_heroes |= set(team)
    if len(free_heroes) > 0:
        team_for_task = optimize_team_for_task(row, list(free_heroes))
        print(f"Оптимальная команда для поручения {row['Номер поручения']}:", team_for_task)
        query.append((datetime.strptime(row['Дата поручения'], '%Y-%m-%d'), team_for_task))
        teams.append([row['Номер поручения'], team_for_task])
        for name in team_for_task:
            free_heroes -= set(team_for_task)
    else:
        print('Закончились герои')
        break

Оптимальная команда для поручения 11134: ['Агата', 'Бендер', 'Пастушок', 'Мартин']
Оптимальная команда для поручения 11056: ['Синеглазый', 'Юлия', 'Соня']
Оптимальная команда для поручения 11381: ['Агата']
Оптимальная команда для поручения 11311: ['Синеглазый', 'Пастушок', 'Соня']
Оптимальная команда для поручения 11417: ['Синеглазый', 'Пастушок', 'Соня']
Оптимальная команда для поручения 11310: ['Глюкоза']
Оптимальная команда для поручения 11285: ['Пастушок']
Оптимальная команда для поручения 11218: ['Мартин', 'Фредерик']
Оптимальная команда для поручения 11161: ['Пастушок']
Оптимальная команда для поручения 11387: ['Пастушок']
Оптимальная команда для поручения 11143: ['Агата']
Оптимальная команда для поручения 11232: ['Глюкоза', 'Мартин', 'Фредерик']
Оптимальная команда для поручения 11234: ['Бендер', 'Глюкоза']
Оптимальная команда для поручения 11396: ['Бендер', 'Глюкоза', 'Соня']
Оптимальная команда для поручения 11428: ['Бендер', 'Глюкоза']
Оптимальная команда для поручения 11402:

In [47]:
teams_df = pd.DataFrame(teams, columns=['Номер поручения', 'Предложенная команда'])
teams_df.to_csv('../data/teams.csv', sep=';', index=False)
teams_df

,Номер поручения,Предложенная команда
0,11134,"[Агата, Бендер, Пастушок, Мартин]"
1,11056,"[Синеглазый, Юлия, Соня]"
2,11381,[Агата]
3,11311,"[Синеглазый, Пастушок, Соня]"
4,11417,"[Синеглазый, Пастушок, Соня]"
5,11310,[Глюкоза]
6,11285,[Пастушок]
7,11218,"[Мартин, Фредерик]"
8,11161,[Пастушок]
9,11387,[Пастушок]
